# Imports

In [ ]:
%pip install google-colab-selenium

In [ ]:
import google_colab_selenium as gs
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import re
import pandas as pd
from lxml import etree
#import matplotlib.pyplot as plt
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from selenium.webdriver.common.keys import Keys
import math
import pandas as pd
from IPython.display import clear_output
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import json
import datetime
import pandas as pd
from IPython.display import clear_output

In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
48 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

In [ ]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--lang=en")

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set the target URL
url = "put-url-here-to-scrape"

# set up the webdriver


# Functions

In [ ]:
def login(driver):
  driver.get('https://www.facebook.com/login/')

  driver.find_element(By.NAME, "email").send_keys('abdo.barca.boy@hotmail.com')
  print ("Email Id entered")
  sleep(1)

  driver.find_element(By.NAME, "pass").send_keys('modmengame222')
  print ("Password entered")

  driver.find_element(By.NAME,"login").click()

def screenshot(driver):
  driver.get_screenshot_as_file('screen.png')

In [ ]:
def get_head_body(driver):
    '''
    inputs:
        driver(webdriver): The Web Driver will be used

    returns:
        head(etreeELement): Head Etree element
        body(etreeELement): Body Etree element
    '''
    # Getting Head Element
    head=driver.find_element(By.TAG_NAME,'head')
    headElemnt = head.get_attribute('outerHTML') #gives exact HTML content of the element
    headHtml = BeautifulSoup(headElemnt,'html') # Convert the string content into html elements
    headEtree= etree.HTML(str(headHtml)) # Convert the Element to etree elemnt (LXML)

    # Getting Body Elemnt
    body=driver.find_element(By.TAG_NAME,'body')
    bodyElemnt = body.get_attribute('outerHTML') #gives exact HTML content of the element
    bodyHtml = BeautifulSoup(bodyElemnt,'html') # Convert the string content into html elements
    bodyEtree = etree.HTML(str(bodyHtml)) # Convert the Element to etree elemnt (LXML)

    return headEtree , bodyEtree

In [ ]:
def find_text(path):
  element = driver.find_element(By.XPATH,path)
  html_content = element.get_attribute('innerHTML')
  s = BeautifulSoup(html_content, 'html.parser')
  text_elements = s.find_all(text=True)
  cleaned_texts=[]
  for t in text_elements:
    cleaned_texts.append(re.sub(r"[^\w\s]", "", t))

  return cleaned_texts

In [ ]:
def contains_number(text):
  text=str(text)
  return bool(re.search(r'\d', text, flags=re.IGNORECASE))

In [ ]:
IMG_BASE_PATH='/html/body/div[1]/div/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div[2]/div/div/div/div/div/div/div/div/div'

def content_type(body):
  m1=None
  m2=None
  try:
    m1=body.xpath(f'{IMG_BASE_PATH}/div[13]/div/div/div[3]/div[2]/div[1]/div/div/div/div[1]/a/div[1]/div[1]/div/img')
  except:
    pass
  try:
    m2=body.xpath(f'{IMG_BASE_PATH}/div[13]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img')
  except:
    pass
  if len(m1) >0 or len(m2)>0:
    return 'img'
  else:
    return 'other'

In [ ]:
def get_post_elements(driver,url):
    '''
     Get posts elemnts (comments, likes, contetn , creation_date & text)

    inputs:
        utl(str): URL of post page

    '''
    driver.get(url) # Open post page
    sleep(0.5)
    #driver.execute_scrpt("window.scrollTo(0, document.body.scrollHeight);") # executing Script that scroll the page
    head, body= get_head_body(driver) # Get Head & body HTML contents

    content='Other'

    if url.split('/')[4]=='videos':
        # Get text elemnt
        text=find_text('/html/body/div[1]/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/div/div[1]/div/div/div[2]/div[1]/div[5]/div/span')
        # Get likes elemnt
        likes=body.xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/div/div[1]/div/div/div[1]/div[2]/div[2]/div/div/div[2]/div/div[1]/div/span/span/span/text()[1]')
        if not contains_number(likes):
          likes=body.xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/div/div[1]/div/div/div[1]/div[2]/div[2]/div/div/div[2]/div/div[1]/div/span/span/span/text()[2]')
        # Get comments elemnt
        comments=body.xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/div/div[1]/div/div/div[1]/div[2]/div[2]/div/div/div[2]/div/div[3]/span/a/span')[0].text
        # Get Post creation time
        # Get content element
        content= 'vid'

    else:
        # Get text elemnt
        text=find_text(f'{IMG_BASE_PATH}/div[*]/div/div/div[3]/div[1]/div/div/div/span')

        # Get likes elemnt
        likes=body.xpath(f'{IMG_BASE_PATH}/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[1]/div/span/div/span[2]/span/span/text()[1]')
        if not contains_number(likes):
          likes=body.xpath(f'{IMG_BASE_PATH}/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[1]/div/span/div/span[2]/span/span/text()[2]')
        # Get content element
        content=content_type(body)

        try: # comments have some bugs, so some times they break the code
        # Get comments elemnt
            comments=body.xpath(f'{IMG_BASE_PATH}/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/span/a/div/div[1]/span')


            comments=comments[0].text
        except:
          try:
            sleep(0.5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # executing Script that scroll the page
            head, body= get_head_body(driver) # Get Head & body HTML contents
            comments=body.xpath(f'{IMG_BASE_PATH}/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/span/a/div/div[1]/span')

            comments=comments[0].text
          except:
            pass

    time_script=find_creation_time_script(body)
    time_unix=find_creation_time(time_script)
    date= convert_unix_time(time_unix)
    full_text=""
    for line in text:
        full_text+=line
    return {'text':full_text,'content':content,'date':date,'comments':comments,'likes':likes,'link':url}


In [ ]:
def find_creation_time_script(body):
  for i in range(400):
    x=body.xpath(f'/html/body/script[{i}]')

    try:
      if "creation_time" in x[0].text:
        return x[0].text
    except:
      pass

In [ ]:
def find_creation_time(data):
  return int(data.split("creation_time")[1].split('\"')[1].replace(':','').replace(',',''))

In [ ]:
def convert_unix_time(date):
  dt = datetime.datetime.fromtimestamp(date)
  # Format the datetime as a string
  formatted_date = dt.strftime("%Y-%m-%d")
  return formatted_date

In [ ]:
def convert_seconds(seconds):
  """Converts seconds to hours, minutes, and seconds."""
  hours = int(seconds // 3600)
  minutes = int((seconds % 3600) // 60)
  seconds = int(seconds % 60)
  return f"{hours} hours, {minutes} minutes, {seconds} seconds"



# Main

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
login(driver)

Email Id entered
Password entered


In [ ]:
screenshot(driver)

In [ ]:
df = pd.read_csv('/content/Khalilalbalush_Links (2).csv',index_col=False)
# df_done1=pd.read_csv('/content/Talha_Data (1).csv',index_col=False)
# df_done2=pd.read_csv('/content/Awwad_Data2.csv',index_col=False)

In [ ]:
links=list(df['link'])
links_set=set()
# links_set = set(list(df_done1['link']))
# links_set.update(list(list(df_done2['link'])))
dataset=pd.DataFrame(columns=['text','content','date','comments','likes','link'])
# print(len(links_set))
remaining_links = list(set(links) - links_set)
len(remaining_links)

1296

In [ ]:
errors=0
done=0
full_time=0
expeced_time =0
remaining=len(links)-len(links_set)-errors
for i in remaining_links:
  clear_output()

  print('Progress: ' , done, '| remaining: ',remaining , '                 errors: ' , errors)
  print('Time left: ' ,convert_seconds(expeced_time))
  try:
    if i not in links_set:
      start_time = time.time()
      row=get_post_elements(driver,i)
      end_time = time.time()
      dataset.loc[len(dataset)]=row
      links_set.add(i)

  except Exception as e:
    print(e)
    errors+=1

  if done % 10 == 0:
     dataset.to_csv(f'Khalil_Data.csv', index=False)
  remaining-=1
  done+=1
  full_time+=(end_time - start_time)
  expeced_time = (full_time/done) * remaining
dataset.to_csv(f'Khalil_Data.csv', index=False)

Progress:  454 | remaining:  842                  errors:  7
Time left:  1 hours, 49 minutes, 13 seconds


In [ ]:
 full_time=0
expeced_time =0
remaining=5
for i in range(5):
  start_time = time.time()
  driver.get('https://www.facebook.com/DerradjiHafid/posts/pfbid0jjiEKx7PXtjjS8ETsK7G4JQJwU8xqaigmTPwYbuSQJcwd7uLNfFx8sXp8HeK1TCql?__cft__[0]=AZUilD2rNVR0_Jid6h7k8X6x3t3lsRG5p8beAasJ87382G6zrC_VQ-GrlOWyiM-lM0IngXUpShdEwgEx1pCbK1Kj6opLzLKr1YQRPvDdORG3ndsRDfNS541p0ynmCagFPizn8_N_ZaXbQEMyk8cXRFKG9LM8sJWka7MDtVT6ex51OiaoAPbOn2QcYSvV1BKnXiOvcbAAVPw4Xq257WF_bqic&__tn__=%2CO%2CP-R')
  screenshot(driver)
  end_time = time.time()
  full_time+=(end_time - start_time)
  remaining-=1
  expeced_time = (full_time/(i+1)) * remaining
  print('expected time: ', convert_seconds(expeced_time))

expected time:  0 hours, 0 minutes, 33 seconds
expected time:  0 hours, 0 minutes, 19 seconds
expected time:  0 hours, 0 minutes, 11 seconds
expected time:  0 hours, 0 minutes, 5 seconds
expected time:  0 hours, 0 minutes, 0 seconds


In [ ]:

from google.colab import files

files.download('/content/Khalil_Data.csv')

In [ ]:
df1=pd.read_csv('/content/Awwad_Data1.csv',index_col=False)
df2=pd.read_csv('/content/Awwad_Data2.csv',index_col=False)
df3=pd.read_csv('/content/Awwad_Data3.csv',index_col=False)
print(df1.shape[0]+df2.shape[0]+df3.shape[0])


3037


In [ ]:
df4=pd.concat([df1,df2,df3],axis=0)
df4.shape

(3037, 6)

In [ ]:
df4.to_csv('Awwad_Data.csv',index=False)